In [ ]:
import os
import time
import pandas as pd
from dotenv import load_dotenv

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re

from transformers import pipeline

In [ ]:
def start_driver():
    options = Options()
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    service = Service('chromedriver.exe')  # Assumes chromedriver.exe is in the same folder
    driver = webdriver.Chrome(service=service, options=options)
    
    return driver

In [ ]:
driver = start_driver()


In [ ]:
driver.get('https://www.tiktok.com/messages?lang=pt-BR')



In [ ]:
driver.find_element(By.XPATH, "//*[@id=\"more-acton-icon-0\"]/div/div[2]/p[1]").click()

In [ ]:
# Step 1: Get all chat messages
messages = driver.find_elements(By.XPATH, '//div[@data-e2e="chat-item"]')

print(f"🔎 Found {len(messages)} chat messages.")



In [ ]:
scraped_data = []

messages = messages[:5]
# Step 2: Loop through messages
for idx, msg in enumerate(messages):
    try:
        # Check if it's from @quimerai
        sender_link = msg.find_element(By.XPATH, './/a[contains(@href, "/@quimerai")]')
        
        if sender_link:
            try:
                video_div = msg.find_element(By.XPATH, './/div[contains(@class, "DivVideoContainer")]')
                if video_div:
                    # Click the video thumbnail
                    driver.execute_script("arguments[0].click();", video_div)
                    time.sleep(3)  # Wait for popup to start appearing

                    # Wait for the modal to appear
                    try:
                        wait = WebDriverWait(driver, 10)
                        modal = wait.until(
                            EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "DivContentContainer")]'))
                        )

                        # Try to get description inside modal
                        description = None

                        try:
                            description_element = modal.find_element(
                                By.XPATH,
                                '//div[@data-e2e="browse-video-desc"]//span'
                            )
                            description = description_element.text.strip()
                        except:
                            try:
                                description_element = modal.find_element(
                                    By.XPATH,
                                    './/div[contains(@class, "css-") and contains(text(), "#")]'
                                )
                                description = description_element.text.strip()
                            except:
                                pass

                        if description:
                            hashtags = [word for word in description.split() if word.startswith('#')]
                            usernames = [word for word in description.split() if word.startswith('@')]
                            video_url = driver.current_url

                            # Store collected data
                            scraped_data.append({
                                "Video URL": video_url,
                                "Description": description,
                                "Hashtags": ', '.join(hashtags),
                                "Usernames": ', '.join(usernames)
                            })

                    except Exception as e:
                        pass  # Modal did not appear properly

                    # Close video view (press ESC)
                    body = driver.find_element(By.TAG_NAME, 'body')
                    body.send_keys(Keys.ESCAPE)
                    time.sleep(2)

            except Exception as e:
                pass  # Error processing video
    except Exception as e:
        pass  # Not a message from @quimerai

# ✅ Save results to CSV
if scraped_data:
    df = pd.DataFrame(scraped_data)
    print("✅ Data saved to tiktok_scraped_videos.csv")
else:
    print("⚠️ No data to save.")

In [ ]:
driver.get('https://www.instagram.com/direct/inbox/')



In [ ]:
driver.get('https://www.instagram.com/direct/t/100616051343884/')

In [ ]:
time.sleep(2)

video_data = []  # To store video URL and description

# Find all messages
messages = driver.find_elements(By.XPATH, '//div[contains(@aria-label, "Mensagens na conversa com Isabella Marques")]')
#messages = driver.find_elements(By.XPATH, '//div[contains(@role, "log")]')

In [ ]:
messages[0].find_elements(By.TAG_NAME, 'img')[4].get_attribute('class')

In [ ]:
thumbnails = messages[0].find_elements(By.TAG_NAME, 'img')
#filter tumbnails
final_thumbnails = []
for x in thumbnails:
    chosen_class = x.get_attribute('class')
    chosen_class = chosen_class.split()
    if "x1iyjqo2" in chosen_class or "xz74otr" in chosen_class:
        final_thumbnails.append(x)

print(f"🔎 Found {len(thumbnails)} thumbnails. but only used {len(final_thumbnails)} thumbnails")

for idx, thumb in enumerate(final_thumbnails):
    try:
        print(f"➡️ Clicking thumbnail {idx+1}...")
        driver.execute_script("arguments[0].click();", thumb)
        time.sleep(2)

        # 3. Wait for modal with video to appear
        wait = WebDriverWait(driver, 5)
        video = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'video')))

        video_src = video.get_attribute('src')
        print(f"🎥 Video URL: {video_src}")

        # 4. (Optional) Try to find a description near the video
        try:
            description_element = driver.find_element(By.XPATH, '//h1[contains(@class, "_ap3a")]')  # Adjust if needed
            description = description_element.text
            print(f"📝 Description: {description}")
        except:
            print("⚠️ No description found.")

        # Close modal (press ESC)
        from selenium.webdriver.common.keys import Keys
        body = driver.find_element(By.TAG_NAME, 'body')
        body.send_keys(Keys.ESCAPE)
        time.sleep(2)

    except Exception as e:
        print(f"❌ Error processing thumbnail {idx+1}: {e}")
        continue
print("🎯 Done!")

In [ ]:
# # Step 2: Loop through messages
# for idx, msg in enumerate(messages):
#     try:
#         # Check if it's from @quimerai
#         sender_link = msg.find_element(By.XPATH, './/a[contains(@href, "/@quimerai")]')
        
#         if sender_link:
#             print(f"\n📩 Message {idx+1} is from @quimerai.")
            
#             # Check if it's a video
#             try:
#                 video_div = msg.find_element(By.XPATH, './/div[contains(@class, "DivVideoContainer")]')
#                 if video_div:
#                     print("🎬 Video detected. Clicking...")
                    
#                     # Click the video thumbnail
#                     driver.execute_script("arguments[0].click();", video_div)
#                     time.sleep(3)  # Wait for popup to start appearing

#                     # ✅ Step 2.1: Wait for the modal to appear
#                     try:
#                         wait = WebDriverWait(driver, 10)
#                         modal = wait.until(
#                             EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "DivContentContainer")]'))
#                         )
#                         print("✅ Modal opened!")

#                         # ✅ Step 2.2: Try to get description inside modal
#                         description = None

#                         try:
#                             description_element = modal.find_element(
#                                 By.XPATH,
#                                 '//div[@data-e2e="browse-video-desc"]//span'
#                             )
#                             description = description_element.text.strip()
#                             # description_element = modal.find_element(By.XPATH, './/div[contains(@data-e2e, "browse-video-desc")]')
#                             # description = description_element.text
#                         except:
#                             # Try div fallback
#                             try:
#                                 description_element = modal.find_element(By.XPATH, './/div[contains(@class, "css-") and contains(text(), "#")]')
#                                 description = description_element.text
#                             except:
#                                 print("⚠️ Could not find description element.")

#                         if description:
#                             print(f"📝 Description: {description}")

#                             # Extract hashtags
#                             hashtags = [word for word in description.split() if word.startswith('#')]
#                             print(f"🏷️ Hashtags: {hashtags}")

#                             # Extract usernames mentioned
#                             usernames = [word for word in description.split() if word.startswith('@')]
#                             print(f"👤 Usernames: {usernames}")

#                             # Get video URL (current URL)
#                             video_url = driver.current_url
#                             print(f"🔗 Video URL: {video_url}")

#                             print("-" * 40)
#                         else:
#                             print("❌ No description found inside modal.")

#                     except Exception as e:
#                         print(f"❌ Modal did not appear properly: {e}")

#                     # Close video view (press ESC)
#                     body = driver.find_element(By.TAG_NAME, 'body')
#                     body.send_keys(Keys.ESCAPE)
#                     time.sleep(2)

#                 else:
#                     print("⚠️ No video container found in this message.")
#             except Exception as e:
#                 print(f"❌ Error processing message {idx+1}: {e}")
#     except Exception as e:
#         pass

# print("\n🎯 All done!")
# #driver.quit()


In [ ]:
df.to_csv('tiktok_scraped_videos.csv', index=False, encoding='utf-8-sig')